In [1]:
import os
import tensorflow
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import argparse
import locale
import sys
!pip install hdf5storage
import hdf5storage as hdf5

sys.path.insert(0,"/content/drive/My Drive/Hyperspectral")

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
os.chdir("/content/gdrive/MyDrive/Hyperspectral/")
print(os.getcwd())
!pip install hdf5storage
import datasets
import models
import cv2

     |████████████████████████████████| 53 kB 1.7 MB/s 
Mounted at /content/gdrive
/content/gdrive/MyDrive/Hyperspectral


In [2]:
# load firmness
#inputPath = "c:\\users\\nalina\\Documents\\tomato_firmness\\hsdata\\firmness.csv"
#inputPath = "/content/gdrive/MyDrive/Hyperspectral/hsdata50/firmness_hs50_omit.csv"
inputPath = "/content/gdrive/MyDrive/Hyperspectral/hsdata50/firmness_hs50.csv"
df = datasets.load_firmness_attributes(inputPath)
df.shape

(371, 1)

In [1]:
#load the hyperspectral images of tomatoes (50 channels) and scale the pixel 
inputPath = "/content/gdrive/MyDrive/Hyperspectral/hsdata50/"
#images = datasets.load_produce_images(df_sub, inputPath)
inputImages = []
fileCount = 1
for i in df.index.values:
  filename = "reconCube" + i[1:] + ".txt"
  img = np.loadtxt(inputPath  + filename)
  img = img.reshape((50,88,88))
  img = img.reshape(88,88,50)
  #img_reshaped = img.reshape(img.shape[0], -1)
  #print(img_reshaped.shape)
  #np.savetxt((img_path + i + ".txt"), img_reshaped, fmt='%d')
  #img = spectral[i, :, :]
  filenum = int(filename.partition('.')[0][9:])
  #if (filenum >= 11 and filenum <= 392):
  #  img = img*0.5
  #else:
  #  img = img*0.2
  inputImages.append(img)
  fileCount += 1





NameError: ignored

In [4]:
inputImages = np.array(inputImages)
#inputImages1 = inputImages.reshape
inputImages.shape

(371, 88, 88, 50)

In [5]:
split = train_test_split(df, inputImages, test_size=0.25, random_state=42)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split

trainY = np.log(trainAttrX["firmness"])
testY = np.log(testAttrX["firmness"])
print(trainImagesX.shape)

(278, 88, 88, 50)


In [6]:
# create our Convolutional Neural Network and then compile the model
# using mean absolute percentage error as our loss, implying that we
# seek to minimize the absolute percentage difference between our
# predicted firmness and the actual firmness
model = models.create_cnn(88, 88, 50, regress=True)
opt = Adam(lr=1e-4, decay=1e-3 / 200)
model.compile(loss="mean_squared_error", optimizer=opt)

history = model.fit(x=trainImagesX, y=trainY, validation_data=(testImagesX, testY), epochs=200, batch_size=32) #try epochs=200 if this works, error goes down until 125

#firmness predictions for test data
#firmness_preds = model.predict(testImagesX)

#compute the difference between the firmness predictions and the actual firmness values, then compute the percentage 
#difference and the absolute percentage difference
#diff = firmness_preds.flatten() - testY
#percent_diff = (diff / testY) * 100
#absPercentDiff = np.abs(percent_diff)

#mean and std deviation of absolute percentage difference
#mean = np.mean(absPercentDiff)
#std = np.abs(absPercentDiff)
#######################################

#print("Avg. firmness " + str(df["firmness"].mean()))
#print("Std. Dev. firmness " + str(df["firmness"].std()))

#print("Mean and std. dev. of predictions " + str(mean) + " " + str(std))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/200
9/9 [==============================] - 5s 408ms/step - loss: 3.4073 - val_loss: 4.1149
Epoch 2/200
9/9 [==============================] - 3s 379ms/step - loss: 2.8291 - val_loss: 6.6432
Epoch 3/200
9/9 [==============================] - 3s 372ms/step - loss: 2.6359 - val_loss: 6.8513
Epoch 4/200
9/9 [==============================] - 3s 376ms/step - loss: 2.6349 - val_loss: 6.2040
Epoch 5/200
9/9 [==============================] - 3s 373ms/step - loss: 2.6128 - val_loss: 5.5196
Epoch 6/200
9/9 [==============================] - 3s 378ms/step - loss: 2.1917 - val_loss: 5.0299
Epoch 7/200
9/9 [==============================] - 3s 385ms/step - loss: 2.2695 - val_loss: 4.2371
Epoch 8/200
9/9 [==============================] - 3s 376ms/step - loss: 2.1627 - val_loss: 3.8248
Epoch 9/200
9/9 [==============================] - 3s 378ms/step - loss: 1.9434 - val_loss: 3.6706
Epoch 10/200
9/9 [==============================] - 3s 371ms/step - loss: 1.7953 - val_loss: 3.5111
Epoch 11/

In [3]:
import matplotlib.pyplot as plt

print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: ignored

In [4]:
import pandas as pd
#testY = testAttrX["firmness"] / maxFirmness
#firmness predictions for test data
firmness_preds = model.predict(testImagesX)
testY = pd.DataFrame(testY)
testY['preds'] = firmness_preds.flatten()
testY.to_csv("/content/gdrive/MyDrive/Hyperspectral/hsdata50/test_preds_200epochs_v4.csv", index=False)

NameError: ignored

In [9]:
d = {'train_loss': history.history['loss'], 'test_loss': history.history['val_loss']}
model_acc = pd.DataFrame(d)
model_acc.to_csv("/content/gdrive/MyDrive/Hyperspectral/hsdata50/model_acc_200epochs_v4.csv", index=False)

In [ ]:
inputNNImages = inputImages
inputNNImages.shape

In [ ]:
split = train_test_split(df, inputNNImages, test_size=0.25, random_state=42)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split

trainY = np.log(trainAttrX["firmness"])
testY = np.log(testAttrX["firmness"])

In [ ]:
#flatten the trainImagesX and testImagesX to convert from 3 dimensions to 1 dimension
nn_trainx = trainImagesX.reshape(trainImagesX.shape[0],-1)
nn_testx = testImagesX.reshape(testImagesX.shape[0],-1)
print(nn_trainx.shape)


In [ ]:
# create our Neural Network and then compile the model
# using mean absolute percentage error as our loss, implying that we
# seek to minimize the absolute percentage difference between our
# predicted firmness and the actual firmness
model = models.create_mlp(387200, regress=True)
opt = Adam(lr=1e-4, decay=1e-3 / 200)
model.compile(loss="mean_squared_error", optimizer=opt)

nn_history = model.fit(x=nn_trainx, y=trainY, validation_data=(nn_testx, testY), epochs=125, batch_size=32) #try epochs=200 if this works

#firmness predictions for test data
nn_firmness_preds = model.predict(nn_testx)

#compute the difference between the firmness predictions and the actual firmness values, then compute the percentage 
#difference and the absolute percentage difference
#nn_diff = firmness_preds.flatten() - testY
#nn_percent_diff = (nn_diff / testY) * 100
#nn_absPercentDiff = np.abs(nn_percent_diff)

#mean and std deviation of absolute percentage difference
#nn_mean = np.mean(nn_absPercentDiff)
#nn_std = np.abs(nn_absPercentDiff)

In [ ]:
import matplotlib.pyplot as plt

print(nn_history.history.keys())
# summarize history for accuracy
plt.plot(nn_history.history['loss'])
plt.plot(nn_history.history['val_loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
import pandas as pd
#testY = testAttrX["firmness"] / maxFirmness
#firmness predictions for test data
nn_firmness_preds = model.predict(nn_testx)
testY = pd.DataFrame(testY)
testY['preds'] = nn_firmness_preds.flatten()
testY.to_csv("/content/gdrive/MyDrive/Hyperspectral/hsdata50/nn_test_preds_125epochs_v1.csv", index=False)

In [ ]:
d = {'train_loss': nn_history.history['loss'], 'test_loss': nn_history.history['val_loss']}
model_acc = pd.DataFrame(d)
model_acc.to_csv("/content/gdrive/MyDrive/Hyperspectral/hsdata/nn_model_acc_125epochs_v1.csv", index=False)